# Create profiles of your ribs


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

def plot_excel_sheets(file_path, prominence=0.0001):
    # Read all sheets into a dictionary of DataFrames.
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    
    # Determine global x-axis limits across all sheets.
    global_x_min = float('inf')
    global_x_max = float('-inf')
    for sheet_name, df in all_sheets.items():
        if 'distance' in df.columns:
            global_x_min = min(global_x_min, df['distance'].min())
            global_x_max = max(global_x_max, df['distance'].max())
    
    # Determine the greatest z-range among sheets.
    max_z_range = 0
    for sheet_name, df in all_sheets.items():
        if 'z' in df.columns:
            sheet_z_range = df['z'].max() - df['z'].min()
            max_z_range = max(max_z_range, sheet_z_range)
    
    # Use a minimum y-range of 20.
    global_y_range = max(max_z_range, 20)
    
    # Create a directory to export the plot files.
    export_dir = "plots_exported"
    os.makedirs(export_dir, exist_ok=True)
    
    # Plot each sheet.
    for sheet_name, df in all_sheets.items():
        if 'distance' in df.columns and 'z' in df.columns:
            plt.figure(figsize=(10, 6))
            
            # Detect dips ("ribs") using find_peaks on the negative of z.
            dips_indices, properties = find_peaks(-df['z'], prominence=prominence)
            
            # First, plot the red dots for dips with a lower zorder.
            plt.scatter(df['distance'].iloc[dips_indices], df['z'].iloc[dips_indices], 
                        color='yellow', zorder=2, label='Rib (dip)')
            
            # Then, plot the main line in black with a higher zorder.
            plt.plot(df['distance'], df['z'], linestyle='-', color='black', zorder=3, label='Profile')
            
            plt.xlabel('Distance')
            plt.ylabel('Z Height')
            plt.title(f'Graph for Sheet: {sheet_name}')
            plt.grid(True)
            plt.xlim(global_x_min, global_x_max)
            
            # Center the y-axis on the curve's midpoint.
            z_min, z_max = df['z'].min(), df['z'].max()
            mid = (z_min + z_max) / 2
            plt.ylim(mid - global_y_range / 2, mid + global_y_range / 2)
            
            plt.legend()
            
            # Define file paths for SVG and PDF.
            svg_path = os.path.join(export_dir, f"{sheet_name}_plot.svg")
            pdf_path = os.path.join(export_dir, f"{sheet_name}_plot.pdf")
            
            # Save the plot as SVG and PDF.
            plt.savefig(svg_path)
            plt.savefig(pdf_path)
            
            plt.show()  # Display the plot.
            plt.close() # Close the figure.
        else:
            print(f"Sheet '{sheet_name}' does not have the required columns 'distance' and 'z'.")

if __name__ == '__main__':
    file_path = input("Please enter the path to your Excel file: ")
    plot_excel_sheets(file_path, prominence=0.0001)


# Get a excel file of each rib that is marked on the plot

In [ ]:
import os
import pandas as pd
from scipy.signal import find_peaks

def export_rib_data_from_excel(file_path, prominence=0.0001, export_dir="exported_csv"):
    """
    Reads an Excel file, detects rib dips in each sheet based on the 'z' values,
    and exports a CSV file for each profile containing the location (distance) and height (z) of each rib.
    
    Parameters:
    - file_path: str, path to the Excel file.
    - prominence: float, prominence parameter for find_peaks function.
    - export_dir: str, directory where the CSV files will be saved.
    """
    # Read all sheets into a dictionary of DataFrames.
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    
    # Create the export directory if it doesn't exist.
    os.makedirs(export_dir, exist_ok=True)
    
    # Process each sheet.
    for sheet_name, df in all_sheets.items():
        if 'distance' in df.columns and 'z' in df.columns:
            # Detect dips ("ribs") using find_peaks on the negative of z.
            dips_indices, properties = find_peaks(-df['z'], prominence=prominence)
            
            if len(dips_indices) > 0:
                # Extract rib data.
                rib_locations = df['distance'].iloc[dips_indices]
                rib_heights = df['z'].iloc[dips_indices]
                rib_df = pd.DataFrame({
                    "Location of rib": rib_locations.values,
                    "Height of rib": rib_heights.values
                })
                
                # Save the rib data to a CSV file.
                csv_path = os.path.join(export_dir, f"{sheet_name}_ribs.csv")
                rib_df.to_csv(csv_path, index=False)
                print(f"CSV for sheet '{sheet_name}' saved to: {csv_path}")
            else:
                print(f"No ribs detected in sheet '{sheet_name}'. No CSV created.")
        else:
            print(f"Sheet '{sheet_name}' does not have the required columns 'distance' and 'z'.")

# Example usage:
if __name__ == '__main__':
    file_path = input("Enter the path to your Excel file: ")
    export_rib_data_from_excel(file_path, prominence=0.0001)


# Get rib number, latitude, longitude, water depth, rib height, cumulative distance of profile, and distance between rib

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

def refine_rib_coordinate(s_series, z_series, index):
    """
    Refines the rib's horizontal location (s coordinate) by performing quadratic interpolation on
    three points (s, z) around the detected dip index.
    
    To improve numerical stability, the s values are centered using their middle value.
    If the index is at a boundary or an error occurs, returns the original s value.
    """
    if index > 0 and index < len(s_series) - 1:
        s_vals = s_series.iloc[index-1:index+2].values
        z_vals = z_series.iloc[index-1:index+2].values
        s_center = s_vals[1]
        s_local = s_vals - s_center
        try:
            coeffs = np.polyfit(s_local, z_vals, 2)
            a, b, _ = coeffs
            if a != 0:
                vertex_local = -b / (2 * a)
                if s_local[0] <= vertex_local <= s_local[2]:
                    return vertex_local + s_center
        except Exception:
            pass
    return s_series.iloc[index]

def interpolate_lat_long(refined_s, s_series, lat_series, lon_series):
    """
    Given a refined horizontal location (refined_s), linearly interpolates the
    latitude and longitude along the profile.
    """
    if refined_s <= s_series.iloc[0]:
        return lat_series.iloc[0], lon_series.iloc[0]
    if refined_s >= s_series.iloc[-1]:
        return lat_series.iloc[-1], lon_series.iloc[-1]
    
    for i in range(len(s_series) - 1):
        if s_series.iloc[i] <= refined_s <= s_series.iloc[i+1]:
            t = (refined_s - s_series.iloc[i]) / (s_series.iloc[i+1] - s_series.iloc[i])
            lat_interp = lat_series.iloc[i] + t * (lat_series.iloc[i+1] - lat_series.iloc[i])
            lon_interp = lon_series.iloc[i] + t * (lon_series.iloc[i+1] - lon_series.iloc[i])
            return lat_interp, lon_interp
    return lat_series.iloc[-1], lon_series.iloc[-1]

def export_rib_data_from_excel(file_path, prominence=0.0001, export_dir="exported_excel"):
    """
    Reads an Excel file containing profile data with at least the following columns:
       - "distance":  The horizontal distance (cumulative) along the profile.
       - "lat":       The latitude at each sampled point.
       - "z":         The water depth or elevation at each sampled point.
       - "lon" or "long":  The longitude at each sampled point.
    
    For each sheet, the code:
      - Uses the provided "distance" column as the horizontal coordinate.
      - Detects rib dips in the elevation by finding peaks in (-z).
      - Refines each rib’s horizontal location via quadratic interpolation using the "distance" column.
      - Interpolates latitude and longitude for the refined rib position.
      - Uses the rib’s prominence as the rib height.
      - Computes the spacing between successive refined rib locations.
      - Numbers each rib sequentially.
    
    The output is a single Excel workbook with one sheet per valid profile (suffixed with "_ribs").
    Each output sheet contains:
        - Rib Number (sequentially numbered)
        - Latitude
        - Longitude
        - Z at rib (water depth or elevation)
        - Height of rib (prominence)
        - Distance at rib
        - Spacing to Previous Rib
    
    If the required columns are missing or no ribs are detected, a message is written instead.
    """
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    os.makedirs(export_dir, exist_ok=True)
    output_excel_path = os.path.join(export_dir, "ribs.xlsx")
    
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        for sheet_name, df in all_sheets.items():
            if 'distance' in df.columns and 'lat' in df.columns and 'z' in df.columns and ('lon' in df.columns or 'long' in df.columns):
                lon_col = 'lon' if 'lon' in df.columns else 'long'
                s_series = df["distance"]
                dips_indices, properties = find_peaks(-df['z'], prominence=prominence)
                
                if len(dips_indices) > 0:
                    prominences = properties["prominences"]
                    refined_s_list = []
                    refined_coords_lat = []
                    refined_coords_lon = []
                    z_at_rib = []
                    
                    for idx in dips_indices:
                        refined_s = refine_rib_coordinate(s_series, df['z'], idx)
                        lat_interp, lon_interp = interpolate_lat_long(refined_s, s_series, df['lat'], df[lon_col])
                        refined_s_list.append(refined_s)
                        z_at_rib.append(df['z'].iloc[idx])
                        refined_coords_lat.append(lat_interp)
                        refined_coords_lon.append(lon_interp)
                    
                    spacing = [None] + [refined_s_list[i] - refined_s_list[i-1] for i in range(1, len(refined_s_list))]
                    
                    rib_df = pd.DataFrame({
                        "Latitude": refined_coords_lat,
                        "Longitude": refined_coords_lon,
                        "Z at rib": z_at_rib,
                        "Height of rib": prominences,
                        "Distance at rib": refined_s_list,
                        "Spacing to Previous Rib": spacing
                    })
                    
                    # Add the sequential rib number as the first column.
                    rib_df.insert(0, "Rib Number", range(1, len(rib_df) + 1))
                    
                    export_sheet_name = f"{sheet_name}_ribs"[:31]
                    rib_df.to_excel(writer, sheet_name=export_sheet_name, index=False)
                    print(f"Sheet '{sheet_name}' processed: {len(dips_indices)} ribs detected and exported as '{export_sheet_name}'.")
                else:
                    no_ribs_df = pd.DataFrame({"Message": ["No ribs detected"]})
                    no_ribs_df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"No ribs detected in sheet '{sheet_name}'.")
            else:
                missing_cols_df = pd.DataFrame({"Message": ["Missing required columns: 'distance', 'lat', 'z', and ('lon' or 'long')"]})
                missing_cols_df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"Sheet '{sheet_name}' is missing required columns.")
    
    print(f"\nAll done! Rib data exported to '{output_excel_path}'.")

if __name__ == '__main__':
    file_path = input("Enter the path to your Excel file: ")
    export_rib_data_from_excel(file_path, prominence=0.0001)


# Plot profiles with slope from QGIS

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

def find_matching_column(df, keywords):
    """
    Returns the first column name whose lower-case name contains any of the keywords.
    """
    for col in df.columns:
        lower_col = col.lower()
        for kw in keywords:
            if kw in lower_col:
                return col
    return None

def plot_profiles_and_slopes(profiles_file, slopes_file, prominence=0.0001):
    # Read profiles file (all sheets) and slopes file (all sheets)
    profiles_dict = pd.read_excel(profiles_file, sheet_name=None)
    slopes_dict = pd.read_excel(slopes_file, sheet_name=None)
    
    # Normalize column names to lower case for all sheets.
    for sheet_name, df in profiles_dict.items():
        df.columns = [c.lower() for c in df.columns]
    for sheet_name, df in slopes_dict.items():
        df.columns = [c.lower() for c in df.columns]
    
    # Calculate global x-axis limits and overall elevation range for profiles.
    global_x_min = np.inf
    global_x_max = -np.inf
    global_z_range = 0
    for sheet_name, df in profiles_dict.items():
        dist_col = find_matching_column(df, ['dist', 'distance'])
        z_col = find_matching_column(df, ['z', 'elev', 'height'])
        if dist_col is None or z_col is None:
            print(f"Skipping sheet '{sheet_name}' due to missing required columns.")
            continue
        global_x_min = min(global_x_min, df[dist_col].min())
        global_x_max = max(global_x_max, df[dist_col].max())
        global_z_range = max(global_z_range, df[z_col].max() - df[z_col].min())
    global_y_range = max(global_z_range, 20)
    
    # --- New: Compute global slope limits ---
    global_slope_min = np.inf
    global_slope_max = -np.inf
    # Loop over each slopes sheet.
    for sheet_name, slope_df in slopes_dict.items():
        # Look for an exact match of "slope_value"
        slope_value_col = None
        for col in slope_df.columns:
            if col.lower() == 'slope_value':
                slope_value_col = col
                break
        if slope_value_col is not None and not slope_df.empty:
            cur_min = slope_df[slope_value_col].min()
            cur_max = slope_df[slope_value_col].max()
            if cur_min < global_slope_min:
                global_slope_min = cur_min
            if cur_max > global_slope_max:
                global_slope_max = cur_max
    if global_slope_min == np.inf or global_slope_max == -np.inf:
        print("No slope_value data found in slopes file; slope axis will not be scaled.")
        global_slope_range = 0
    else:
        global_slope_range = global_slope_max - global_slope_min
        slope_midpoint = (global_slope_min + global_slope_max) / 2

    # Create directory for exported plots.
    export_dir = "plots_exported"
    os.makedirs(export_dir, exist_ok=True)
    
    # Loop over each profile sheet.
    for sheet_name, profile_df in profiles_dict.items():
        dist_col = find_matching_column(profile_df, ['dist', 'distance'])
        z_col = find_matching_column(profile_df, ['z', 'elev', 'height'])
        if dist_col is None or z_col is None:
            print(f"Skipping profile sheet '{sheet_name}' due to missing required columns.")
            continue
        
        fig, ax1 = plt.subplots(figsize=(10, 3))
        # Plot the profile elevation.
        ax1.plot(profile_df[dist_col], profile_df[z_col], color='black', label='Profile')
        # Identify dips (ribs) using find_peaks on negative elevation.
        dips_indices, _ = find_peaks(-profile_df[z_col], prominence=prominence)
        ax1.scatter(profile_df[dist_col].iloc[dips_indices],
                    profile_df[z_col].iloc[dips_indices],
                    color='yellow', label='Rib')
        
        ax1.set_xlabel('Distance')
        ax1.set_ylabel('Elevation')
        ax1.set_title(f'Profile: {sheet_name}')
        ax1.grid(True)
        ax1.set_xlim(global_x_min, global_x_max)
        mid = (profile_df[z_col].min() + profile_df[z_col].max()) / 2
        ax1.set_ylim(mid - global_y_range/2, mid + global_y_range/2)
        
        # Look for a sheet in the slopes file with the same name as the current profile.
        slope_sheet = None
        for key in slopes_dict:
            if key.lower() == sheet_name.lower():
                slope_sheet = slopes_dict[key]
                break
        
        if slope_sheet is None:
            print(f"No matching slope sheet found for profile '{sheet_name}'.")
        else:
            # In the slope sheet, look for the "slope_value" column (exact match ignoring case).
            slope_value_col = None
            for col in slope_sheet.columns:
                if col.lower() == 'slope_value':
                    slope_value_col = col
                    break
            if slope_value_col is None:
                print(f"In slopes sheet '{sheet_name}', column 'slope_value' not found.")
            else:
                # Also, try to identify a distance column in the slopes sheet.
                slope_distance_col = find_matching_column(slope_sheet, ['dist', 'distance'])
                if slope_distance_col is None:
                    print(f"In slopes sheet '{sheet_name}', no distance column found.")
                else:
                    ax2 = ax1.twinx()
                    ax2.plot(slope_sheet[slope_distance_col], slope_sheet[slope_value_col],
                             linestyle='--', color='grey', label='Slope (QGIS)')
                    ax2.set_ylabel('Slope')
                    # Apply global slope limits if available.
                    if global_slope_range > 0:
                        ax2.set_ylim(slope_midpoint - global_slope_range/2, slope_midpoint + global_slope_range/2)
                    # Combine legends from both axes.
                    lines1, labels1 = ax1.get_legend_handles_labels()
                    lines2, labels2 = ax2.get_legend_handles_labels()
                    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
        
        # Save and display the plot.
        svg_path = os.path.join(export_dir, f"{sheet_name}_plot.svg")
        pdf_path = os.path.join(export_dir, f"{sheet_name}_plot.pdf")
        plt.savefig(svg_path)
        plt.savefig(pdf_path)
        plt.show()
        plt.close()

if __name__ == '__main__':
    profiles_file = input("Enter full path to your rib profiles Excel file: ")
    slopes_file = input("Enter full path to your slopes Excel file: ")
    plot_profiles_and_slopes(profiles_file, slopes_file, prominence=0.0001)
